# Ejemplo de uso del mecanismo de deteccion de anomalias

A continuacion se presenta un ejemplo de como se puede usar el mecanismo de deteccion de anomalias cargando los modelos que guardamos en el Notebook MecanismoDeteccionAnomalias

In [3]:
# file backend/server/apps/ml/income_classifier/random_forest.py
import joblib
import pandas as pd
import numpy as np
import keras

class AnomalyDetector:

    __instance = None

    def __init__(self):
        path_to_artifacts = ""
        self.scaler =  joblib.load(path_to_artifacts + "scaler.joblib")
        self.pca = joblib.load(path_to_artifacts + "pca.joblib")
        #self.model = joblib.load(path_to_artifacts + "model_nn3.joblib")
        self.model = keras.models.load_model(path_to_artifacts + "model_nn3.h5")
        self.if_model = joblib.load(path_to_artifacts + "isolation_model.joblib")

    def __new__(cls):
        if AnomalyDetector.__instance is None:
            AnomalyDetector.__instance = object.__new__(cls)
        return AnomalyDetector.__instance

    def preprocessing(self, input_data):
        # JSON to pandas DataFrame
        input_data = pd.read_json(input_data, orient='records')
        # input_data = pd.DataFrame(input_data, index=[0])
        # fill missing values
        input_data = input_data.values
        input_data = self.scaler.transform(input_data)
        input_data = self.pca.transform(input_data)
        return np.array([input_data])

    def predict(self, input_data):
        diff = self.model.predict(input_data) - input_data
        return self.if_model.predict(diff[0].reshape((1,9)))

    def postprocessing(self, input_data):
        label = "Normal"
        if input_data[0] < 1:
            label = "Anomaly"
        return {"probability": input_data[0], "label": label, "status": "OK"}

    def compute_prediction(self, input_data):
        try:
            input_data = self.preprocessing(input_data)
            print('pred')
            print(input_data.shape)
            prediction = self.predict(input_data)
            print('post')
            prediction = self.postprocessing(prediction)
        except Exception as e:
            return {"status": "Error", "message": str(e)}

        return prediction

Using TensorFlow backend.


In [4]:
ad = AnomalyDetector()

In [5]:
ad.compute_prediction('[{"acc_x":4.194016,"acc_y":-0.726532, "acc_z":8.979568,"gyr_x":-0.000900,"gyr_y":-0.000748, "gyr_z":0.000748},{"acc_x":4.206207,"acc_y":-0.716492,"acc_z":8.832672,"gyr_x":-0.000732,"gyr_y":-0.000351,"gyr_z": 0.000870},{"acc_x":4.175949,"acc_y":-0.676086,"acc_z":8.834686,"gyr_x":0.000061,"gyr_y":0.000183,"gyr_z":-0.000458}]')

pred
(1, 3, 3)
post


{'probability': 1, 'label': 'Normal', 'status': 'OK'}

In [6]:
ad.compute_prediction('[{"acc_x":4.798996,"acc_y":0.255478, "acc_z":6.945023,"gyr_x":0.039642,"gyr_y":0.007202, "gyr_z":0.053223	},{"acc_x":5.240753,"acc_y":0.271469,"acc_z":7.980225,"gyr_x":0.001465,"gyr_y":-0.008545,"gyr_z": 0.012909},{"acc_x":3.782837,"acc_y":2.956512,"acc_z":8.834686,"gyr_x":0.141922,"gyr_y":0.033630,"gyr_z":0.275391}]')

pred
(1, 3, 3)
post


{'probability': -1, 'label': 'Anomaly', 'status': 'OK'}